In [2]:
import csv
from collections import defaultdict
from collections import OrderedDict
import re


RACES = ['Australia', 'Malaysia', 'China', 'Bahrain',
         'Spain', 'Monaco', 'Canada', 'Austria',
         'England', 'Hungary', 'Belgium', 'Italy', 
         'Singapore', 'Japan', 'Russia', 'America',
         'Mexico', 'Brazil' , 'Abu Dhabi']

DRIVERS = {'Rosberg':'Nico Rosberg', 'Hamilton':'Lewis Hamilton', 'Raikkonen':'Kimi Räikkönen',
           'Perez':'Sergio Pérez', 'Ricciardo':'Daniel Ricciardo', 'Bottas':'Valtteri Bottas',
          'Hulkenberg':'Nico Hülkenberg', 'Massa':'Felipe Massa', 'Kvyat':'Daniil Kvyat', 
          'Sainz':'Carlos Sainz Jr.', 'Verstappen':'Max Verstappen', 'Button':'Jenson Button',
          'Maldonado':'Pastor Maldonado', 'Nasr':'Felipe Nasr', 'Grosjean':'Romain Grosjean',
          'Vettel':'Sebastian Vettel', 'Alonso':'Fernando Alonso', 'Ericsson':'Marcus Ericsson',
          'Stevens':'Will Stevens', 'Merhi':'Roberto Merhi', 'Rossi':'Alexander Rossi'}

Fantasy_Finish_Points_Dict = defaultdict(int)
for position in range(1,11):
    Fantasy_Finish_Points_Dict[position] = 11 - position

class Team():
    def __init__(self, name):
        global RACES
        new_driver = False
        if name not in RACES:
            print(name, 'is an invalid race name.\nPlease select from the following:\n', RACES)
        self.name = name
        with open('Teams/team_' + self.name + '.csv', 'rt', newline="\r\n") as f:
            teams = [line.replace('"', '').split(',') for line in f if len(line.split(',')) > 1]
            teams.pop(0) #remove header
        pattern = re.compile('^(.*?)\(')
        for line in range(len(teams)):
            for driver in range(2, len(teams[line])):
                teams[line][driver] = pattern.search(teams[line][driver].strip()).group(1).strip()
        self.teams = {}
        for line in teams:
            self.teams[line[1]] = line[2::]
            
    def drop(self, team, racer):
        if team not in self.teams:
            print(team, 'is not in the league.  Try', self.name + '.teams to see your options.')
            raise
        if racer not in self.teams[team]:
            print(racer, 'is not on', team + "'s team.  Try", self.name + '.teams to see the current teams.')
            raise
        self.teams[team].remove(racer)
        self.push()
        
    def push(self):
        #Push self.teams to the csv
        header = [['Player', 'Driver1', 'Driver2', 'Driver3', 'Driver4', 'Driver5']]
        for player in self.teams.keys():
            header.append([player])
            header[header.index([player])].extend(self.teams[player])     
        with open('teams/team_' + self.name + '.csv', 'wt', newline='') as f:
            csv_writer = csv.writer(f)
            csv_writer.writerows(header)
        
    def add(self, team, racer):
        if team not in self.teams:
            print(team, 'is not in the league.  Try', self.name + '.teams to see your options.')
            raise
        if len(self.teams[team]) == 5:
            print(team + "'s team already has five drivers.  Consider using .drop or .replace")
        self.teams[team].append(racer)  
        self.push()
        
    def replace(self, team, drop_racer, add_racer):
        if team not in self.teams:
            print(team, 'is not in the league.  Try', self.name + '.teams to see your options.')
            raise
        if drop_racer not in self.teams[team]:
            print(drop_racer, 'is not on', team + "'s team.  Try", self.name + '.teams to see the current teams.')
            raise
        self.teams[team].remove(drop_racer)
        self.teams[team].append(add_racer)
        self.push()
        
    def __repr__(self):
        print_string = ''
        for team in self.teams:
            print_string += team + ':\t' + str(sorted(self.teams[team])) + '\n'
        return print_string
    
    def validate(self, team):
        global DRIVERS
        driver_cost = {}
        cost_column = RACES.index(self.name) * 2 + 2
        with open('Season_Drivers.csv', 'rt', newline='') as f:
            for line in f:
                val = line.split(',')
                driver_cost[val[0]] = val[cost_column]
        budget = 33
        for driver in self.teams[team]:
            if driver not in DRIVERS.values():
                driver = DRIVERS[driver]
            budget -= int(driver_cost[driver])
        return budget
    
class Race():
    """Race is an object that correlates to a specific F1 race.  It contains results for 
    qualifying, grid position, finish positions, and the corresponding scores for the fantasy
    league, along with methods for updating any of the positions.
    """
    def __init__(self, name):
        global RACES
        global DRIVERS
        if name not in RACES:
            print(name, 'is an invalid race name.\nPlease select from the following:\n', RACES)
            raise
        self.name = name
        with open('Results/' + self.name + '_Qualifying.csv', 'rt', newline='') as f:
            qualifying_results = [line.split(',') for line in f if len(line.split(',')) > 1]
            qualifying_results.pop(0) #remove header
        with open('Results/' + self.name + '_Race.csv', 'rt', newline='') as f:
            race_results = [line.split(',') for line in f if len(line.split(',')) > 1]
            race_results.pop(0) #remove header
        self.fastest_lap = race_results.pop()[1].strip()
        self.lap_length = race_results.pop()[1].strip()
        self.drivers = {line[2].strip() for line in qualifying_results}
        self.drivers_position = {}
        self.drivers_points = {}
        self.drivers_cost = {}
        self.grid_position = {}
        self.qualifying_position = {}
        self.constructor_finish = {}
        self.drivers_team = {}
        for line in race_results:
            if line[7].strip() == '':
                self.drivers_points[line[2]] = 0
            else:
                self.drivers_points[line[2]] = int(line[7].strip())
            if line[0].isdigit():
                self.drivers_position[line[2]] = int(line[0])
            else:
                self.drivers_position[line[2]] = line[0]
            try:
                self.grid_position[line[2]] = int(line[6])
            except:
                self.grid_position[line[2]] = max(self.grid_position.values()) + 1
            if line[3].strip() not in self.constructor_finish and line[0].strip().isdigit():
                self.constructor_finish[line[3]] = [int(line[0])]
            elif line[0].strip().isdigit():
                self.constructor_finish[line[3]].append(int(line[0]))
            else:
                self.constructor_finish[line[3]] = [0]
            self.drivers_team[line[2]] = line[3]
        for line in qualifying_results:
            try:
                self.qualifying_position[line[2]] = int(line[0])
            except:
                self.qualifying_position[line[2]] = "DNF"
        self.fantasy_points = {}
        for line in race_results:
            driver = line[2]
            #Award points for finishing in the top 10
            self.fantasy_points[driver] = Fantasy_Finish_Points_Dict[self.drivers_position[driver]]
            #Award points for completing the race
            if int(line[4])/int(self.lap_length) > .9:
                self.fantasy_points[driver] += 3
            elif int(line[4])/int(self.lap_length) > .5:
                self.fantasy_points[driver] += 1
        for driver in self.fantasy_points:
            #Award bonus point for being the fastest on the team
            if self.drivers_position[driver] == min(self.constructor_finish[self.drivers_team[driver]]):
                self.fantasy_points[driver] += 1
            #Award movement bonus points
            if type(self.drivers_position[driver]) == int and type(self.grid_position[driver]) == int:
                if self.grid_position[driver] > self.drivers_position[driver] and self.drivers_position[driver] <= 10:
                    self.fantasy_points[driver] += min(self.grid_position[driver] - self.drivers_position[driver], 10)
        for line in qualifying_results:
            driver = line[2]
            if line[0].isdigit():
                if int(line[0]) == 1:
                    self.fantasy_points[driver] += 3
                elif int(line[0]) <= 10:
                    self.fantasy_points[driver] += 2
                elif int(line[0]) <= 15:
                    self.fantasy_points[driver] += 1
                
    def __repr__(self):
        ordered_position = OrderedDict(sorted(self.drivers_position.items(), key=lambda t: str(t[1])))
        ordered_position = OrderedDict(sorted(ordered_position.items(), key=lambda t: len(str(t[1]))))
        print_string = ''
        for driver, position in ordered_position.items():
            print_string += str(position) + '\t' + driver + '\n'
        return print_string
    
    def update_grid(self, driver, position):
        if driver in DRIVERS:
            driver = DRIVERS[driver]
        if driver in self.drivers:
            self.grid_position[driver] = position
            if list(self.grid_position.values()).count(position) > 1:
                print("WARNING: Two drivers have the same grid position.  Use", self.name + '.update_grid to correct this.')
            self.print_grid()
            self.push()
        else:
            print(driver, 'is not a valid driver.  See', self.name + '.drivers to see valid names.')
        
    def update_qualifying(self, driver, position):
        if driver in DRIVERS:
            driver = DRIVERS[driver]
        if driver in self.drivers:
            self.qualifying_position[driver] = position
            if list(self.qualifying_position.values()).count(position) > 1:
                print("WARNING: Two drivers have the same qualifying position.  Use", self.name + '.update_qualifying to correct this.')
            self.print_qualifying()
            self.push()
        else:
            print(driver, 'is not a valid driver.  See', self.name + '.drivers to see valid names.')
        
    def update_race(self, driver, position):
        if driver in DRIVERS:
            driver = DRIVERS[driver]
        if driver in self.drivers:
            self.drivers_position[driver] = position
            if list(self.drivers_position.values()).count(position) > 1:
                print("WARNING: Two drivers have the same finish position.  Use", self.name + '.update_finish to correct this.')
            self.print_race()
            self.push()
        else:
            print(driver, 'is not a valid driver.  See', self.name + '.drivers to see valid names.')
            
    def print_race(self):
        ordered_position = OrderedDict(sorted(self.drivers_position.items(), key=lambda t: str(t[1])))
        ordered_position = OrderedDict(sorted(ordered_position.items(), key=lambda t: len(str(t[1]))))
        print_string = ''
        for driver, position in ordered_position.items():
            print_string += str(position) + '\t' + driver + '\n'
        print("Pos. \tFinish Results\n" + print_string)
        
    def print_qualifying(self):
        ordered_position = OrderedDict(sorted(self.qualifying_position.items(), key=lambda t: str(t[1])))
        ordered_position = OrderedDict(sorted(ordered_position.items(), key=lambda t: len(str(t[1]))))
        print_string = ''
        for driver, position in ordered_position.items():
            print_string += str(position) + '\t' + driver + '\n'
        print("Pos. \tQualifying\n" + print_string)
        
    def print_grid(self):
        ordered_position = OrderedDict(sorted(self.grid_position.items(), key=lambda t: str(t[1])))
        ordered_position = OrderedDict(sorted(ordered_position.items(), key=lambda t: len(str(t[1]))))
        print_string = ''
        for driver, position in ordered_position.items():
            print_string += str(position) + '\t' + driver + '\n'
        print("Pos. \tGrid Position\n" + print_string)
        
    def push(self):
        #Push results to the Racing CSV
        with open('results/' + self.name + '_Race.csv', 'rt', newline='') as f:
            reader = csv.reader(f)
            lines = []
            for row in reader:
                lines.append(row)
            for line in lines:
                if len(line) >= 7:
                    if line[2] in self.drivers:
                        line[6] = str(self.grid_position[line[2]])
                        line[0] = str(self.drivers_position[line[2]])
        with open('results/' + self.name + '_Race.csv', 'wt', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(lines)
            
        #Push results to the Qualifying CSV
        with open('results/' + self.name + '_Qualifying.csv', 'rt', newline='') as f:
            reader = csv.reader(f)
            lines = []
            for row in reader:
                lines.append(row)
            for line in lines:
                if len(line) >= 8:
                    if line[2] in self.drivers:
                        line[7] = str(self.grid_position[line[2]])
                        line[0] = str(self.qualifying_position[line[2]])
        with open('results/' + self.name + '_Qualifying.csv', 'wt', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(lines)
    def push_to_season(self, TeamObj):
        #Results have been finalized, append these to the season documents
        with open('Season_Drivers.csv', 'rt', newline='') as f:
            reader = csv.reader(f)
            lines = []
            for row in reader:
                lines.append(row)
        index = RACES.index(self.name)
        last_4_race_sum = 0
        for line in lines[1::]:
            if line[0] not in self.fantasy_points.keys():
                line[index * 2 + 3] = 0
            else:
                line[index * 2 + 3] = self.fantasy_points[line[0]]
            last_4_race_sum += sum(list(map(int, line[max(3, index*2-3):index*2+4:2])))
        #Add costing to the next race column
        if self.name != RACES[-1]:
            for line in lines[1::]:
                prev_scores = list(map(int, line[max(3, index*2-3):index*2+4:2]))
                average_score = sum(prev_scores)/last_4_race_sum*100
                line[index*2 + 4] = max([round(average_score), 1])
                self.drivers_cost[line[0]] = line[index*2]
        
        with open('Season_Drivers.csv', 'wt', newline = '') as f:
            writer = csv.writer(f)
            writer.writerows(lines)
        #Push to Season_Teams.csv
        with open('Season_Teams.csv', 'rt', newline='') as f:
            reader = csv.reader(f)
            lines = []
            for row in reader:
                lines.append(row)
        for row, line in enumerate(lines[1::]):
            player = line[0]
            if (row + 1)%6 > 0 or row == 0:
                driver = TeamObj.teams[player][row%6-1]
                line[index*2 + 1] = driver
                line[index*2 + 2] = self.fantasy_points[driver]
            else:
                line[index*2 + 1] = 'Total:'
                line[index*2 + 2] = sum([self.fantasy_points[driver] for driver in TeamObj.teams[player]]) + TeamObj.validate(player)
        with open('Season_Teams.csv', 'wt', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(lines)

In [8]:
season = []
for race in RACES:
    print(race)
    This_Race = Race(race)
    This_Team = Team(race)
    #This_Race.push_to_season(This_Team)
    season.append(This_Race)

Australia
Malaysia
China
Bahrain
Spain
Monaco
Canada
Austria
England
Hungary
Belgium
Italy
Singapore
Japan
Russia
America
Mexico
Brazil
Abu Dhabi


In [13]:
print(This_Race.drivers_points)

{'Carlos Sainz Jr.': 0, 'Nico Hülkenberg': 6, 'Sebastian Vettel': 12, 'Jenson Button': 0, 'Daniel Ricciardo': 8, 'Nico Rosberg': 25, 'Will Stevens': 0, 'Daniil Kvyat': 1, 'Valtteri Bottas': 0, 'Fernando Alonso': 0, 'Felipe Nasr': 0, 'Max Verstappen': 0, 'Pastor Maldonado': 0, 'Kimi Räikkönen': 15, 'Sergio Pérez': 10, 'Roberto Merhi': 0, 'Lewis Hamilton': 18, 'Felipe Massa': 4, 'Marcus Ericsson': 0, 'Romain Grosjean': 2}
